In [11]:
# Import Libraries
import numpy as np
import json
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input

In [12]:
#Upload VGG16

vgg16 = VGG16(weights='imagenet')

In [13]:
vgg16.save('vgg16.h5')

In [14]:
def prepare_image(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [15]:
y = prepare_image('D:/MACHINE LEARNING WITH P/Indian Number Plates/Image_Car.jpeg')

In [16]:
preds = vgg16.predict(y)

In [17]:
preds

array([[4.93286052e-08, 5.80345954e-07, 1.00345922e-07, 4.21890505e-08,
        1.43857918e-07, 7.03453651e-08, 3.23974803e-08, 1.53356290e-04,
        6.45119580e-05, 1.04430756e-05, 6.64391209e-08, 5.64218148e-08,
        1.08399752e-06, 7.39092698e-08, 3.60579300e-08, 6.31932039e-07,
        2.27719238e-06, 1.02995905e-07, 4.65957896e-07, 5.11743146e-08,
        7.80249678e-08, 6.45736918e-07, 4.47172823e-07, 3.55882403e-05,
        9.24865105e-07, 1.28899001e-06, 1.68273516e-06, 1.26289842e-06,
        3.25018448e-07, 1.10683253e-07, 1.66244803e-07, 1.41648286e-07,
        6.64048230e-07, 2.70163554e-07, 1.83505460e-06, 3.76484968e-06,
        2.54002566e-06, 2.14695774e-06, 2.44424626e-07, 6.97018493e-07,
        4.85892031e-07, 1.61219828e-06, 2.88939759e-06, 2.68039145e-07,
        5.57789235e-07, 1.56459578e-06, 3.70416444e-07, 2.74767075e-07,
        4.76334094e-07, 7.82200516e-07, 1.10328308e-07, 1.03144873e-06,
        5.61607294e-07, 1.43907727e-07, 3.06923226e-07, 6.925522

In [18]:
CLASS_INDEX = None

In [19]:
#CLASS_INDEX_PATH = 'C:/Users/Ashar/Desktop/cdd/imagenet_class_index.json'
CLASS_INDEX_PATH = 'imagenet_class_index.json'

In [20]:
def get_predictions(preds,top=5):
    global CLASS_INDEX
    
    #Load the jason file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    
    #get the results
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [21]:
print (get_predictions(preds, top=5))

[[('n03345487', 'fire_engine', 0.55471784), ('n03594945', 'jeep', 0.16038385), ('n04461696', 'tow_truck', 0.1310885), ('n03930630', 'pickup', 0.12135504), ('n03967562', 'plow', 0.0044336896)]]


In [22]:
#First Check - category list creation

In [23]:
from collections import Counter, defaultdict
import os
import pickle as pk
from shutil import copy

In [32]:
def get_car_categories():
    d = defaultdict(float)
    img_list = os.listdir('D:\\MACHINE LEARNING WITH P\\data1\\training\\whole1')
    for i, img_path in enumerate(img_list):
        img = prepare_image('D:/MACHINE LEARNING WITH P/data1/training\whole1/' + img_path)
        out = vgg16.predict(img)
        top = get_predictions(out, top=5)
        for j in top[0]:
            d[j[0:2]] += j[2]
        if i % 100 == 0:
            print (i, '/', len(img_list), 'complete')
    return Counter(d)

In [33]:
cat_counter=get_car_categories()

0 / 920 complete
100 / 920 complete
200 / 920 complete
300 / 920 complete
400 / 920 complete
500 / 920 complete
600 / 920 complete
700 / 920 complete
800 / 920 complete
900 / 920 complete


In [34]:
#with open('cat_counter.pk', 'wb') as f:
 #   pk.dump(cat_counter,f,-1)

In [35]:
# This piece of code will be used to load already saved cat_counter
#with open('cat_counter.pk', 'rb') as f:
 #   cat_counter = pk.load(f)

In [36]:
cat_list  = [k for k, v in cat_counter.most_common()[:27]]

In [37]:
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n02814533', 'beach_wagon'),
 ('n03930630', 'pickup'),
 ('n03670208', 'limousine'),
 ('n04037443', 'racer'),
 ('n03594945', 'jeep'),
 ('n03769881', 'minibus'),
 ('n04461696', 'tow_truck'),
 ('n03459775', 'grille'),
 ('n02930766', 'cab'),
 ('n02974003', 'car_wheel'),
 ('n03796401', 'moving_van'),
 ('n03977966', 'police_van'),
 ('n04252225', 'snowplow'),
 ('n02701002', 'ambulance'),
 ('n04065272', 'recreational_vehicle'),
 ('n04467665', 'trailer_truck'),
 ('n03445924', 'golfcart'),
 ('n03345487', 'fire_engine'),
 ('n03791053', 'motor_scooter'),
 ('n03776460', 'mobile_home'),
 ('n04252077', 'snowmobile'),
 ('n02704792', 'amphibian'),
 ('n03417042', 'garbage_truck'),
 ('n02965783', 'car_mirror')]

# First Check - car or not

In [38]:
def car_categories_gate(image_path, cat_list):
    img = prepare_image(image_path)
    out = vgg16.predict(img)
    top = get_predictions(out, top=5)
    print ("Validating that this is a picture of your car...")
    for j in top[0]:
        if j[0:2] in cat_list:
            print (j[0:2])
            return "Validation complete - proceed to damage evaluation"
    return "Are you sure this is a picture of your car? Please take another picture (try a different angle or lighting) and try again."

In [39]:
car_categories_gate('C:/Users/KIIT/Desktop/College_ID.jpg', cat_list)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/KIIT/Desktop/College_ID.jpg'